In [1]:
import requests
import re
from bs4 import BeautifulSoup
import json

In [2]:
PTT_URL = 'https://www.ptt.cc/bbs/Gossiping/index.html'

In [3]:
def crawl_article(url):
    response = requests.get(url, cookies = {'over18':'1'})
    if response.status_code != 200:
        print('Error - {} is not available to access'.format(url))

    soup = BeautifulSoup(response.text, 'lxml')

    #內文
    main_content = soup.find(id='main-content')

    #作者、標題、發文時間
    metas = main_content.find_all('div', class_='article-metaline')

    author = metas[0].find('span', class_='article-meta-value').text.strip()
    title = metas[1].find('span', class_='article-meta-value').text.strip()
    date = metas[2].find('span', class_='article-meta-value').text.strip()

    #內文移除作者標題發文時間等
    for m in metas:
        m.extract()
    #內文移除看板資訊
    for m in main_content.find_all('div', class_='article-metaline-right'):
        m.extract()

    #ip
    IP = main_content.find_all('span', class_='f2')
    for i in IP:
        if u'發信站' in i.text:
            ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', i.text).group()
    #推文
    p, b, n = 0, 0, 0
    messages = []
    pushes = main_content.find_all('div', class_='push')
    for push in pushes:
        #main_contain移除push
        push.extract()
        if not push.find('span', class_='push-tag'):
            continue
        # push_tag 判斷是推文, 箭頭還是噓文
        # push_userid 判斷留言的人是誰
        # push_content 判斷留言內容
        # push_ipdatetime 判斷留言日期時間

        push_tag = push.find('span', class_='push-tag').text.strip()
        push_userid = push.find('span', class_='push-userid').text.strip()
        push_content = push.find('span', class_='push-content').text.strip()
        push_ipdatetime = push.find('span', class_='push-ipdatetime').text.strip()

        messages.append({
            'push_tag':push_tag,
            'push_userid':push_userid,
            'push_content':push_content,
            'push_ipdatetime':push_ipdatetime
        })

        if push_tag == '推':
            p += 1
        elif push_tag =='噓':
            b += 1
        else:
            n += 1
        # 統計推噓文
        # count 為推噓文相抵看這篇文章推文還是噓文比較多
        # all 為總共留言數量 
    messages_count = {'all': p+b+n, 'count': p-b, 'push':p, 'boo':b, 'neutral':n }

    #分割※，取出主文
    content = main_content.text.split('※')[0]

    data =[{
        'url':url,
        'article_author':author,
        'article_title':title,
        'article_date':date,
        'article_content':content,
        'ip':ip,
        'messages_count':messages_count,
        'messages':messages    
    }]
    return data


In [4]:
# 對文章列表送出請求並取得列表主體
resp = requests.get(PTT_URL, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)
main_list = soup.find('div', class_='bbs-screen')
all_data = []

# 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
for div in main_list.find_all('div'):
    if 'r-list-sep' in div.get('class'):
        print('Reach the last article')
        break
    if 'r-ent' in div.get('class'):
        article_title = div.find('div', class_='title').text.strip()
        article_URL = 'https://www.ptt.cc' + div.find('a')['href']
        print('{} - {}'.format(article_title, article_URL))
        
        # 呼叫上面寫好的 function 來對文章進行爬蟲
        parse_data = crawl_article(article_URL)
        
        # 將爬完的資料儲存
        all_data.append(parse_data)

C:\Users\judyh\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\judyh\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


[問卦] 八卦台男下限可以有多低？ - https://www.ptt.cc/bbs/Gossiping/M.1588475805.A.559.html
[問卦] 為什麼RG脈衝鋼蛋沒有附贈巨劍啊? - https://www.ptt.cc/bbs/Gossiping/M.1588475831.A.994.html
[爆卦] 南韓軍方監視哨站遭北韓槍彈擊中 - https://www.ptt.cc/bbs/Gossiping/M.1588475914.A.1B4.html
[問卦] 「含扣」到底是什麼意思 - https://www.ptt.cc/bbs/Gossiping/M.1588475925.A.D28.html
Re: [問卦] 不買房可以輕鬆多少 - https://www.ptt.cc/bbs/Gossiping/M.1588475929.A.534.html
[問卦] 去一次菜市場花一千算正常的八卦 - https://www.ptt.cc/bbs/Gossiping/M.1588475934.A.9A3.html
[問卦] 高中是不是該把國文廢掉改上權力分立總論 - https://www.ptt.cc/bbs/Gossiping/M.1588475949.A.B50.html
[新聞]【一線採訪】武漢軍運村變方艙 業主抗議 - https://www.ptt.cc/bbs/Gossiping/M.1588475979.A.417.html
Re: [爆卦] 判決上網了！ - https://www.ptt.cc/bbs/Gossiping/M.1588475983.A.603.html
[問卦] 矮子練壯是不是更顯矮？ - https://www.ptt.cc/bbs/Gossiping/M.1588475992.A.899.html
Re: [新聞] PTT預告「去校門口對小蘿莉動手」 工程師 - https://www.ptt.cc/bbs/Gossiping/M.1588476022.A.776.html
[問卦] 欸欸，大甲人給我進來 - https://www.ptt.cc/bbs/Gossiping/M.1588476119.A.424.html
Re: [新聞] 轟台版打拋肉「噁到不行」！2配料激怒他 - https://www.p

In [5]:
# 將爬完的資訊存成 json 檔案
with open('parse_data.json', 'w+', encoding = 'utf-8') as f:
    json.dump(all_data, f, ensure_ascii=False, indent=4)